[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/klavis-ai/klavis/blob/main/examples/gemini/Use_Klavis_with_Gemini.ipynb)


# Gemini + Klavis AI Integration

This tutorial demonstrates how to use Google's Gemini with function calling with Klavis MCP (Model Context Protocol) servers.


## Prerequisites

- **Google AI API key** - Get at [ai.google.dev](https://ai.google.dev/)
- **Klavis API key** - Get at [klavis.ai](https://klavis.ai/)


In [32]:
# Install the required packages
%pip install -qU google-generativeai klavis

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import google.generativeai as genai
from klavis import Klavis
from klavis.types import McpServerName, ConnectionType, ToolFormat

# Set environment variables
os.environ["GOOGLE_API_KEY"] = "YOUR_GOOGLE_API_KEY"  # Replace with your actual Google AI API key
os.environ["KLAVIS_API_KEY"] = "YOUR_KLAVIS_API_KEY"  # Replace with your actual Klavis API key

# Configure Gemini
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))


/Users/zihaolin/src/klavis/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Case Study 1 : Gemini + YouTube MCP Server


#### Step 1 - Create YouTube MCP Server using Klavis


In [4]:
klavis_client = Klavis(api_key=os.getenv("KLAVIS_API_KEY"))

youtube_mcp_instance = klavis_client.mcp_server.create_server_instance(
    server_name=McpServerName.YOUTUBE,
    user_id="1234",
    platform_name="Klavis",
    connection_type=ConnectionType.STREAMABLE_HTTP,
)

# print(f"🔗 YouTube MCP server created at: {youtube_mcp_instance.server_url}, and the instance id is {youtube_mcp_instance.instance_id}")


#### Step 2 - Create general method to use MCP Server with Gemini


In [34]:

def gemini_with_mcp_server(mcp_server_url: str, user_query: str):
    # Get tools from MCP server
    mcp_server_tools = klavis_client.mcp_server.list_tools(
        server_url=mcp_server_url,
        connection_type=ConnectionType.STREAMABLE_HTTP,
        format=ToolFormat.GEMINI,
    )
    print(mcp_server_tools)
    
    # Initialize Gemini model with tools
    model = genai.GenerativeModel(
        model_name="gemini-2.5-flash",
        tools=mcp_server_tools.tools
    )
    
    # Start chat
    chat = model.start_chat()
    
    # Send initial message
    response = chat.send_message(user_query)
    
    print(response)
    
    # Check if function call is requested
    if response.candidates[0].content.parts[0].function_call:
        function_call = response.candidates[0].content.parts[0].function_call
        function_name = function_call.name
        function_args = dict(function_call.args)
        
        print(f"🔧 Calling: {function_name}, with args: {function_args}")
        
        # Call the MCP server tool
        result = klavis_client.mcp_server.call_tools(
            server_url=mcp_server_url,
            tool_name=function_name,
            tool_args=function_args,
            connection_type=ConnectionType.STREAMABLE_HTTP
        )
        
        # Send function response back to model
        function_response = genai.protos.Part(
            function_response=genai.protos.FunctionResponse(
                name=function_name,
                response={"result": result.result.content[0]['text']}
            )
        )
        
        final_response = chat.send_message([function_response])
        return final_response.text
    else:
        return response.text


#### Step 3 - Summarize your favorite video!


In [35]:
YOUTUBE_VIDEO_URL = "https://www.youtube.com/watch?v=LCEmiRjPEtQ"  # pick a video you like!

result = gemini_with_mcp_server(
    mcp_server_url=youtube_mcp_instance.server_url, 
    user_query=f"Please provide a complete summary of this YouTube video with timestamp: {YOUTUBE_VIDEO_URL}"
)

print(result)

success=True tools=[{'function_declarations': [{'name': 'get_youtube_video_transcript', 'description': "Retrieve the transcript or video details for a given YouTube video. The 'start' time in the transcript is formatted as MM:SS or HH:MM:SS.", 'parameters': {'type': 'object', 'properties': {'url': {'type': 'string', 'description': 'The URL of the YouTube video to retrieve the transcript/subtitles for. (e.g. https://www.youtube.com/watch?v=dQw4w9WgXcQ)'}}, 'required': ['url']}}]}] format=<ToolFormat.GEMINI: 'gemini'> error=None
response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "function_call": {
                  "name": "get_youtube_video_transcript",
                  "args": {
                    "url": "https://www.youtube.com/watch?v=LCEmiRjPEtQ"
                  }
                }
              }
            

✅ Great! You've successfully created an AI agent that uses Gemini's function calling with Klavis MCP servers to summarize YouTube videos!


## Case Study 2 : Gemini + Gmail MCP Server (OAuth needed)


In [11]:
import webbrowser

# Create Gmail MCP server instance
gmail_mcp_server = klavis_client.mcp_server.create_server_instance(
    server_name=McpServerName.GMAIL,
    user_id="1234",
    platform_name="Klavis",
    connection_type=ConnectionType.STREAMABLE_HTTP,
)

# Redirect to Gmail OAuth page for authorization
webbrowser.open(gmail_mcp_server.oauth_url)

print(f"🔐 Opening OAuth authorization for Gmail, if you are not redirected, please open the following URL in your browser: {gmail_mcp_server.oauth_url}")


🔐 Opening OAuth authorization for Gmail, if you are not redirected, please open the following URL in your browser: https://api.klavis.ai/oauth/gmail/authorize?instance_id=d9d482b3-433a-4330-9a8b-9548c0b0a326


In [ ]:
EMAIL_RECIPIENT = "zihaolin@klavis.ai" # Replace with your email
EMAIL_SUBJECT = "Test Gemini + Gmail MCP Server"
EMAIL_BODY = "Hello World from Gemini!"

result = gemini_with_mcp_server(
    mcp_server_url=gmail_mcp_server.server_url, 
    user_query=f"Please send an email to {EMAIL_RECIPIENT} with subject {EMAIL_SUBJECT} and body {EMAIL_BODY}"
)

print(result)

## Summary

This tutorial demonstrated how to integrate Google's Gemini with function calling capabilities with Klavis MCP servers to create powerful AI applications. We covered two practical examples:

**🎥 YouTube Integration**: Built an AI assistant that can automatically summarize YouTube videos by extracting transcripts and providing detailed, timestamped summaries.

**📧 Gmail Integration**: Created an AI-powered email assistant that can send emails through Gmail with OAuth authentication.

### Key Takeaways:
- **Easy Setup**: Klavis MCP servers can be created with just a few lines of code
- **Gemini Compatible**: All tools are formatted for seamless Gemini function calling
- **Versatile**: Support for both simple APIs (YouTube) and OAuth-authenticated services (Gmail)
- **Scalable**: The same pattern can be applied to any of the MCP servers available in Klavis
- **Multimodal Ready**: Gemini's multimodal capabilities can be leveraged for more complex use cases

**Happy building!** 🚀
